<a href="https://colab.research.google.com/github/Risuo/Springboard/blob/master/Capstone_Prototype_Kaggle_Kernel_v_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The purpose of v.0.2 is to work through some of the various helper functions to get a solid grasp on what their purpose is, what they're doing, and how they're intended to be implemented. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install tifffile

     |████████████████████████████████| 133kB 1.1MB/s 
     |████████████████████████████████| 706kB 61.3MB/s 


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf


tf.keras.backend.clear_session()
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0-rc1


In [0]:
from collections import defaultdict
import csv
import sys

import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff

csv.field_size_limit(sys.maxsize);

In [10]:
#img_path = '/content/drive/My Drive/Capstone/Kaggle/Kaggle Data/three_band/6120_2_2.tif'
img_path = '/content/drive/My Drive/Capstone/Kaggle/Kaggle Data/three_band/6110_0_1.tif'

IM_ID = '6120_2_2'
POLY_TYPE = '1'  # buildings

# Load grid size
x_max = y_min = None
for _im_id, _x, _y in csv.reader(open('/content/drive/My Drive/Capstone/Kaggle/Kaggle Data/grid_sizes.csv')):
    if _im_id == IM_ID:
        x_max, y_min = float(_x), float(_y)
        print('x_max, y_min values:', x_max, y_min)
        break

# Load train poly with shapely
train_polygons = None
for _im_id, _poly_type, _poly in csv.reader(open('/content/drive/My Drive/Capstone/Kaggle/Kaggle Data/train_wkt_v4.csv')):
    if _im_id == IM_ID and _poly_type == POLY_TYPE:
        train_polygons = shapely.wkt.loads(_poly)
        break

# Read image with tiff
im_rgb = tiff.imread('/content/drive/My Drive/Capstone/Kaggle/Kaggle Data/three_band/{}.tif'.format(IM_ID)).transpose([1, 2, 0])
im_size = im_rgb.shape[:2]
print(train_polygons)

x_max, y_min values: 0.009188 -0.00904
MULTIPOLYGON (((0.000439 -0.009039999999999999, 0.000438 -0.008999, 0.000637 -0.008985999999999999, 0.000644 -0.009039999999999999, 0.000439 -0.009039999999999999)), ((0.008607999999999999 -0.009039999999999999, 0.008564 -0.008978, 0.008666 -0.008906000000000001, 0.008770999999999999 -0.009039999999999999, 0.008607999999999999 -0.009039999999999999)), ((0.004793 -0.009039999999999999, 0.0048 -0.009032999999999999, 0.004771 -0.008996000000000001, 0.004825 -0.008954, 0.004898 -0.009039999999999999, 0.004793 -0.009039999999999999)), ((0.001432 -0.009039999999999999, 0.00143 -0.009032999999999999, 0.001671 -0.008992, 0.001682 -0.009039999999999999, 0.001432 -0.009039999999999999)), ((0.004424 -0.009039999999999999, 0.004406 -0.009021, 0.004454 -0.008985999999999999, 0.00449 -0.009039999999999999, 0.004424 -0.009039999999999999)), ((0.004142 -0.009039999999999999, 0.004131 -0.009023, 0.004174 -0.009004, 0.004191 -0.009039999999999999, 0.004142 -0.00903

## These are sample methods provided within a stand-alone kernel from Kaggle which is used within a pipeline to load an image, scale the mask, train, and convert the training back into a mask image.

In [11]:
def get_scalers_testing():
    h, w = im_size  # they are flipped so that mask_for_polygons works correctly
    print(h, w)
    w_ = w * (w / (w + 1))
    print(w_)
    h_ = h * (h / (h + 1))
    print(h_)
    return w_ / x_max, h_ / y_min

x_scaler, y_scaler = get_scalers_testing()

train_polygons_scaled = shapely.affinity.scale(
    train_polygons, xfact=x_scaler, yfact=y_scaler, origin=(0, 0, 0))
print(train_polygons_scaled)

3348 3403
3402.000293772033
3347.0002985965957
MULTIPOLYGON (((162.5465965352549 3347.000298596596, 162.1763309395027 3331.820319366235, 235.8591844942082 3327.007155220023, 238.4510436644742 3347.000298596596, 162.5465965352549 3347.000298596596)), ((3187.246248235705 3347.000298596596, 3170.954562022604 3324.045208053124, 3208.721652789338 3297.387683551027, 3247.599540343328 3347.000298596596, 3187.246248235705 3347.000298596596)), ((1774.683000440722 3347.000298596596, 1777.274859610988 3344.408594825558, 1766.537157334172 3330.709589178648, 1786.531499504795 3315.159366552425, 1813.560887994712 3347.000298596596, 1774.683000440722 3347.000298596596)), ((530.2203331172781 3347.000298596596, 529.4798019257736 3344.408594825558, 618.7138105020751 3329.228615595198, 622.7867320553504 3347.000298596596, 530.2203331172781 3347.000298596596)), ((1638.054995608127 3347.000298596596, 1631.390214884586 3339.965674075209, 1649.162963480696 3327.007155220023, 1662.492524927779 3347.0002985965

In [0]:
def mask_for_polygons_testing(polygons):
    img_mask = np.zeros(im_size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask

train_mask = mask_for_polygons_testing(train_polygons_scaled)

In [0]:
def scale_percentile_testing(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float64)
    # Get 2nd and 98th percentile
    mins = np.percentile(matrix, 1, axis=0)
    maxs = np.percentile(matrix, 99, axis=0) - mins
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix

In [0]:
tiff.imshow(255 * scale_percentile(im_rgb[2900:3200,2000:2300]));

In [0]:
def show_mask_testing(m):
    # hack for nice display
    tiff.imshow(255 * np.stack([m, m, m]));
show_mask_testing(train_mask)